In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/anaconda3/envs/torch/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/envs/torch/lib/python3.10/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1ControlEnv-v1", 32, env_kwargs=dict(
    mode='direction+orientation',
))                

/home/user/anaconda3/envs/torch/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/envs/torch/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    batch_size=4096,
    gradient_steps=2,
    buffer_size=1000000,
    policy_kwargs=dict(        
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=3,
    ),
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
model.learn(
    total_timesteps=int(2e6),
    tb_log_name='g1-control-orientation+direction',
    log_interval=100,
    reset_num_timesteps=False,
)

Logging to ./logs/sac/g1-control-orientation+direction_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 104      |
|    ep_rew_mean     | -210     |
| time/              |          |
|    episodes        | 100      |
|    fps             | 299      |
|    time_elapsed    | 42       |
|    total_timesteps | 12672    |
| train/             |          |
|    actor_loss      | -26.6    |
|    critic_loss     | 2.26     |
|    ent_coef        | 0.792    |
|    ent_coef_loss   | -4.59    |
|    learning_rate   | 0.0003   |
|    n_updates       | 784      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 121      |
|    ep_rew_mean     | -201     |
| time/              |          |
|    episodes        | 200      |
|    fps             | 297      |
|    time_elapsed    | 82       |
|    total_timesteps | 24576    |
| train/             |          |
|    actor_loss      | -3

In [5]:
model.save("trained_models/g1-control/direction+orientation")